In [21]:
import alpdesign as alp
from alpdesign.seq import *
from alpdesign.mlp import *
from alpdesign.utils import *
import jax 
from jax_unirep import get_reps
import haiku as hk

In [22]:
def hamming(seq1, seq2): # seq1 and seq2 are of same length
    seqlist1 = list(seq1)
    seqlist2 = list(seq2)
    hamming_distance = 0.
    for i in range(len(seqlist1)):
        if seqlist1[i] == seqlist2[i]:
            continue
        hamming_distance += 1.
    return hamming_distance / len(seqlist1)
    

In [23]:
target_seq = 'GIGAVLKVLTTGLPALISWIKRKRQQ'

seqs = ['GIGAVLKVLKAGLPALIVTLKRKIVQ',
       'PPGATLKKHTTGSVALISWIWARIQQ',
       'GIGAVLKVLTTGLKTLISAAKRKRAA',
       'HAPPVLKVLTTGLAPPLVWIKRKRTH',
       'GIGAVLUIHKLSSVAAWRPPKRKRQQ']

labels = []
for seq in seqs:
    labels.append(hamming(target_seq, seq))
labels = jnp.array(labels)
    
reps = get_reps(seqs)[0]

In [24]:
key = jax.random.PRNGKey(0)
forward = hk.without_apply_rng(hk.transform(alp.model_forward))
params, outs = alp.ensemble_train(key, forward, reps, labels)
init_x = jax.random.normal(key, shape=(1, 1900))
final_vec = alp.bayes_opt(forward, params, init_x, labels)[0]

In [25]:
print(final_vec)

[-1.8844324  0.0517109  1.2715325 ...  0.6919433 -1.1954689 -1.7313558]


In [28]:
batch_size = 8
bag_num = 2
oh_vec = encode_seq(list(target_seq))
logits = jax.random.normal(key, shape=(jnp.shape(oh_vec)))
b_logits = jax.random.normal(key, shape=(batch_size,*jnp.shape(oh_vec)))
#batch_keys = jax.random.split(key, num=batch_size)
#r = jax.random.normal(r_key)
#b = jax.random.normal(b_key)

params = forward_seqprop.init(key, logits)
#forward_seqprop = hk.transform(forward_seqprop)
beam_loss_traces, beam_loss, beam_seqs = pso_train(key, final_vec, b_logits, params, alp.seq.train_seqprop_adam, batch_size, bag_num)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [33]:
gen_seqs = []
for beam_seq in beam_seqs:
    str = ""
    gen_seqs.append(str.join(beam_seq)) 

In [35]:
print(gen_seqs)


['HIIIMIKKRNYVFIYWRSKGHWIWFP', 'CELACNPKYPIVVPESGCHPLWYPCW', 'CELACNAKYVIVFSPDGCSPDWQPPP', 'CCFEIWKLTRKKHYKWRHRPYVWICG']
GIGAVLKVLTTGLPALISWIKRKRQQ
